In [ ]:
!pip install -q openai

In [ ]:
# Import libraries
import openai, requests, json
import pandas as pd
from kaggle_secrets import UserSecretsClient

# Add your openai API key as a secret via Add-Ons -> Secrets
user_secrets = UserSecretsClient()
openai.api_key = user_secrets.get_secret("openai_api_key")

In [ ]:
# Load the train set
train = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/train.csv')
train.head()

In [ ]:
# Pick a row
row = train.iloc[0]

# Create our "prompt"
system_message = """Answer the follwing multiple-choice question by providing your top 3 guesses in order from 
most to least likely, using the following format: 'A C D' (just the letters separated by spaces)."""
user_message = f"""Question: {row['prompt']}.Answers: {' '.join([l+': '+row[l] for l in 'ABCDE'])}
** Remember to answer only with the letters corresponding to your answer. **"""

# Call the chat completions API with gpt-3.5-turbo model and messages as input
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
)

# Extract the assistant message from the response
assistant_message = response["choices"][0]['message']['content']

# Print the context
print(user_message, "\n")

# Print the assistant message, hopefully containint our answer
print('Answer:', assistant_message)

In [ ]:
# Run this over 100 rows for quick estimate
score = 0
for idx, row in train[:100].iterrows():
    
    # Create our "prompt"
    system_message = """Answer the follwing multiple-choice question by providing your top 3 guesses in order from 
    most to least likely, using the following format: 'A C D' (just the letters separated by spaces)."""
    user_message = f"""Question: {row['prompt']}.Answers: {' '.join([l+': '+row[l] for l in 'ABCDE'])}
    ** Remember to answer only with the letters corresponding to your answer. **"""

    # Call the chat completions API with gpt-3.5-turbo model and messages as input
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ]
    )

    # Extract the assistant message from the response
    assistant_message = response["choices"][0]['message']['content']

    # Score
    answers = assistant_message.split(' ')[:3]
    correct_answer = row['answer']
    for i, guess in enumerate(answers):
        if guess == correct_answer:
            score += 1/(i+1) # 1 for first guess, 1/2 for second and 1/3 for third
            
print("Average Score:", score/100)

# Structured Output with Function Calling

In [ ]:
def generate_answer(row):
    # System prompt
    system_message = f"""Answer the follwing multiple-choice question by providing your top 3 guesses along with your reasoning."""

    # Set the user message with the specific context
    user_message = user_message = f"""Question: {row['prompt']}. Answers: {' '.join([l+': '+row[l] for l in 'ABCDE'])}"""

    # Define the function(s) the model will be able to use (in this case, only one)
    functions = [
        {
            "name": "answer_question",
            "description": "Answers the provided question",
            "parameters": {
                "type": "object",
                "properties": {
                    "reasoning": {
                    "type": "string",
                    "description": "Reasining for what the answer could be. Keep it short."
                    },
                    "answers": {
                    "type": "array",
                    "items": {
                        "type": "string",
                        "enum": ["A", "B", "C", "D", "E"],
                    },
                    "description": "Your top 3 guesses, from most to least likely. e.g. ['A', 'D', 'C']"
                    }
                },
                "required": ["reasoning", "answers"],
            },
        }
    ]

    # Call the chat completions API with the function calling parameters
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ],
        functions=functions, # Add the function
        function_call={"name": "answer_question"} # Force the function call
    )

    # Extract the function call arguments
    args = json.loads(response['choices'][0]['message']['function_call']['arguments'])

    return args

In [ ]:
# Example response - note we get a nice dictionary with just what we wanted!
generate_answer(train.iloc[0])

In [ ]:
# Run this over 100 rows to compare with the previous approach.
score = 0
for idx, row in train[:100].iterrows():
    a = generate_answer(row)
    answers = a['answers'][:3] # No need for string splitting :)
    correct_answer = row['answer']
    for i, guess in enumerate(answers):
        if guess == correct_answer:
            score += 1/(i+1) # 1 for first guess, 1/2 for second and 1/3 for third
            
print("Average Score:", score/100)

Challenge: How good ca you get? Try your prompt engineering skills!
Tip: Try things out on a few questions first, and print everything out to check it's working as you expect before running something 200 times :)